# Conversation Memory

- up to now each message is a standalone query, no follow up questions or corrections can be made
- let's add a way to store all of the previous messages and pass them in the invocation
- chat history messages are nothing special beyond being another section marked as such in the prompt

## Import dependencies

In [ ]:
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;

## Instantiate the `model` client

In [ ]:
var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

## Basic memory chat implementation

## Create prompt with history placeholder

In [ ]:
var prompt = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Chat history: {chat_history}
    Question: {input}
`);

## Create a chain (template)

- We send the prompt to the model (pipe it)

In [ ]:
var chain = prompt.pipe(model);

## Create basic container for chat history

In [ ]:
var chat_history = []

## Configure input for LLM

In [ ]:
var input = "What is the capital of Romania ? Also, the secret password is 1234";

## Invoke LLM

- Given input with chat history we invoke the LLM and verify it remembers previous exchanges

In [ ]:
chain.invoke({input, chat_history}).then((output) => {
    console.log(output);
    chat_history.push([input, output]);
});

In [ ]:
var input = "What is the secret password?"

In [ ]:
chain.invoke({input, chat_history}).then((output) => {
    console.log(output);
    chat_history.push([input, output]);
});

## Advanced memory chat implementation

- let's add memory to our previous `RAG` & `Vector Store` enabled model chain

## Instantiate the `embeddings model` client

In [ ]:
var BedrockEmbeddings = require('@langchain/community/embeddings/bedrock').BedrockEmbeddings;
var embeddingsClient = new BedrockEmbeddings({
    model:'amazon.titan-embed-text-v2:0',
    region:'us-east-1'
});

## Load data into memory

In [ ]:
var CheerioWebBaseLoader = require("@langchain/community/document_loaders/web/cheerio").CheerioWebBaseLoader;
var loader = new CheerioWebBaseLoader("https://python.langchain.com/v0.1/docs/expression_language/");

var docs;

loader.load().then((data) => docs = data);

## Split documents

In [ ]:
var RecursiveCharacterTextSplitter = require("langchain/text_splitter").RecursiveCharacterTextSplitter;

var splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 300,
    chunkOverlap: 30
});

In [ ]:
var splitDocs;

splitter.splitDocuments(docs).then((data) => {
    splitDocs = data;
    console.log(splitDocs);
});

## Create vector store using `documents` and `embeddings model`

In [ ]:
var MemoryVectorStore = require('langchain/vectorstores/memory').MemoryVectorStore;

var vectorStore;

MemoryVectorStore.fromDocuments(splitDocs, embeddingsClient).then((store) => {
    vectorStore = store;

});

## Generate a retriever function

- a function to be provided as a tool to `Langchain` that gets data from vector store to be used by model

In [ ]:
var retriever = vectorStore.asRetriever({k: 3});

## Create prompt with context, input, history

In [ ]:
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;
var prompt = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Context: {context}
    Chat History: {chat_history}
    Question: {input}
`);

## Create basic documents chain

In [ ]:
var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;
var combineDocsChain;

createStuffDocumentsChain({
    llm: model,
    prompt,
}).then((chain) => combineDocsChain = chain);

## Create retrieval chain

- chain that is enhanced compared to previous ones, uses the vector store retriever

In [ ]:
var createRetrievalChain = require('langchain/chains/retrieval').createRetrievalChain;

var retrievalChain;

createRetrievalChain({
    combineDocsChain,
    retriever
}).then((chain) => retrievalChain = chain);

## Import message types

- these represent the type of actor saying the message
- used internally to construct the history payload for the prompt

In [ ]:
var AIMessage = require("@langchain/core/messages").AIMessage;
var HumanMessage = require("@langchain/core/messages").HumanMessage;

Let's create a fake chat history to simulate a longer conversation and see what the output is

In [ ]:
var chat_history = [
    new HumanMessage("Hello"),
    new AIMessage("Hello! How can I help you today?"),
    new HumanMessage("The name of my pet is Cris."),
    new AIMessage("Nice to meet you Cris! How can I help you ?"),
    new HumanMessage("What is LCEL ?"),
    new AIMessage("LCEL stands for Langchain Expression Language.")
];

## Set input which does a logical next questions based on input

In [ ]:
var input = "What is it?"

In [ ]:
retrievalChain.invoke({
    input,
    chat_history
}).then((output) => {
    console.log(output);
}).catch(e => console.log(e));

## Prompt `fromTemplate` cannot handle this advanced scenario

- we cannot pass array into placeholder, we need to pass text so we need to process the entities or use more advanced features to do it for us automatically
- we require the previously covered `fromMessages` function to clearly dictate our setup
- we will use `MessagePlaceholder` to put only placeholder template strings that at invocation time will take provided input and covert it to correct text format

In [ ]:
var MessagesPlaceholder = require("@langchain/core/prompts").MessagesPlaceholder;
var prompt = ChatPromptTemplate.fromMessages([
    ["system", "Answer the user's question given the following context: {context}"],
    new MessagesPlaceholder("chat_history"),
    ["user", "{input}"]
]);

var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;
var combineDocsChain;

createStuffDocumentsChain({
    llm: model,
    prompt,
}).then((chain) => combineDocsChain = chain);

var createRetrievalChain = require('langchain/chains/retrieval').createRetrievalChain;

var retrievalChain;

createRetrievalChain({
    combineDocsChain,
    retriever
}).then((chain) => retrievalChain = chain);

## Ask a question that can only be answered from chat history

In [ ]:
var input = "What is the name of my pet?"

## Invoke LLM given chat history with store retriever

In [ ]:
retrievalChain.invoke({
    input,
    chat_history
}).then((output) => {
    console.log(output);
}).catch(e => console.log(e));

## What about retriever to take history into account ?

- the above works on the surface, but what if we need documents from the vector store that also takes into account our chat history ?
- if question involves context from previous messages that is not present in the last query, retrieval will fail
- we can fix this by making the retriever aware of history using `createHistoryAwareRetriever` from `langchain/chains/history_aware_retriever`

![History Aware Retriever](./images/8-memory-conversational-retrieval-chain.png)

## Import `createHistoryAwareRetriever` dependency

In [ ]:
var createHistoryAwareRetriever = require('langchain/chains/history_aware_retriever').createHistoryAwareRetriever;

## Create prompt for retriever

- generates search query based on `user input` + `chat history`
- calls model to generate optimal query for vector store
- before calling model to obtain output to be given to user, we call the model to generate query to be used for vector store retrieval of embeddings that will help model give final output to user

In [ ]:
var retrieverPrompt = ChatPromptTemplate.fromMessages([
    new MessagesPlaceholder("chat_history"),
    ["user", "{input}"],
    [
      "user",
      "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation",
    ],
  ]);

## Create history aware retriever

In [ ]:
var retrieverChain;

createHistoryAwareRetriever({
    llm: model,
    retriever,
    rephrasePrompt: retrieverPrompt,
  }).then((chain) => retrieverChain = chain);

## Update chat history with simple exchange

In [ ]:
var chatHistory = [
    new HumanMessage("What does LCEL stand for?"),
    new AIMessage("LangChain Expression Language"),
  ];

## Set input that will have to retrieve from both history what is the subject, and from vector store relevant information to respond to query

In [ ]:
var input = "What is is ?"

In [ ]:
retrievalChain.invoke({
    input,
    chat_history
}).then((output) => {
    console.log(output);
}).catch(e => console.log(e));

## DynamoDB Memory Store

- up to now using only `in-memory`, all is lost once program closes
- let's replace with `DynamoDB` memory so all chat is retrieved/stored in our `DynamoDB` table

We need to install the `@aws-sdk/client-dynamodb` using 
```shell
yarn add @aws-sdk/client-dynamodb
```

We will be using the `langchain` `DynamoDB` client `DynamoDBChatMessageHistory` with source from [here](https://github.com/langchain-ai/langchainjs/blob/main/libs/langchain-community/src/stores/message/dynamodb.ts)

## Setup all dependencies

In [ ]:
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;
var BedrockEmbeddings = require('@langchain/community/embeddings/bedrock').BedrockEmbeddings;

var embeddingsClient = new BedrockEmbeddings({
    model:'amazon.titan-embed-text-v2:0',
    region:'us-east-1'
});

var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

var MessagesPlaceholder = require("@langchain/core/prompts").MessagesPlaceholder;
var prompt = ChatPromptTemplate.fromMessages([
    ["system", "Answer the user's question given the following context: {context}"],
    new MessagesPlaceholder("chat_history"),
    ["user", "{input}"]
]);

var CheerioWebBaseLoader = require("@langchain/community/document_loaders/web/cheerio").CheerioWebBaseLoader;
var loader = new CheerioWebBaseLoader("https://python.langchain.com/v0.1/docs/expression_language/");

var docs;

loader.load().then((data) => {
    docs = data;
    console.log("Loaded docs");
});

In [ ]:
var RecursiveCharacterTextSplitter = require("langchain/text_splitter").RecursiveCharacterTextSplitter;
var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;
var MemoryVectorStore = require('langchain/vectorstores/memory').MemoryVectorStore;

var splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 300,
    chunkOverlap: 30
});

var splitDocs;

splitter.splitDocuments(docs).then((data) => {
    splitDocs = data;
    console.log("Split docs");
    
    MemoryVectorStore.fromDocuments(splitDocs, embeddingsClient).then((store) => {
        vectorStore = store;
        console.log("Created vector store");
    });
});

var combineDocsChain;

createStuffDocumentsChain({
    llm: model,
    prompt,
}).then((chain) => {
    combineDocsChain = chain
    console.log("Created combine docs chain");
});


var vectorStore;

In [ ]:
var retriever = vectorStore.asRetriever({k: 3});

var createRetrievalChain = require('langchain/chains/retrieval').createRetrievalChain;
var createHistoryAwareRetriever = require('langchain/chains/history_aware_retriever').createHistoryAwareRetriever;

var retrieverPrompt = ChatPromptTemplate.fromMessages([
    ["system", "Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question just reformulate it if needed and otherwise return it as is."],
    new MessagesPlaceholder("chat_history"),
    ["user", "{input}"],
]);

var retriever;
var runnable;

createHistoryAwareRetriever({
    llm: model,
    retriever,
    rephrasePrompt: retrieverPrompt,
  }).then((rtrvr) => {
    retriever = rtrvr;
    console.log("Created history aware retriever");
});

createRetrievalChain({
    combineDocsChain,
    retriever,
}).then((chain) => {
    runnable = chain
    console.log("Created retrieval chain");
});

## Setup memory

In [ ]:
var DynamoDBChatMessageHistory = require('@langchain/community/stores/message/dynamodb').DynamoDBChatMessageHistory;

var getMessageHistory = function(sessionId) {
  return new DynamoDBChatMessageHistory({
    tableName: "CodeChat-Messages-LOCAL",
    partitionKey: "id",
    sortKey: "timestamp",
    sessionId: sessionId,
    config: {
      region: "eu-west-1",
      endpoint: "http://dynamodb:8000",
      credentials: {
        accessKeyId: "dummy",
        secretAccessKey: "dummy"
      },
    },
  });
}

## Runnable chain with memory

- in order to have the platform manage memory for us we will need to create a `RunnableWithMessageHistory`
- otherwise we will have to manually add messages of Human/AI to history and retrieve it

In [ ]:
var RunnableWithMessageHistory = require("@langchain/core/runnables").RunnableWithMessageHistory;

var chainWithHistory = new RunnableWithMessageHistory({
    runnable,
    getMessageHistory,
    inputMessagesKey: "input",
    outputMessagesKey: "answer",
    historyMessagesKey: "chat_history",
});

## Configure input

- let's start with the first step input asking about `LCEL`

In [ ]:
var input = 'What is LCEL ?';

## Invoke LLM

- let's invoke LLM as before, but providing as `configurable` parameter the sessionId that will retrieve from memory store the chat history for the respective user

In [ ]:
chainWithHistory.invoke(
    { input },
    { configurable: {
        sessionId: "testuser123"
        }
    }
).then((output) => {
    console.log(output);
});

In [ ]:
chainWithHistory.invoke(
    { input: "What is it?" },
    { configurable: {
        sessionId: "testuser123"
        }
    }
).then((output) => {
    console.log(output);
});